In [37]:
import numpy as np
import torch

def coarsening_node_feat(node_feat, source, target): # Idea...
  source_node_feat = node_feat[source]
  target_node_feat = node_feat[target]
  return source_node_feat + target_node_feat

def coarsening_edge_attr(edge_list): # Idea...
  
  return edge_list.sum(axis=0)

In [42]:
node_feat = np.array([[0,0,0,0],[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4],[5,5,5,5]])
edge_index = np.array([[0,0,1,2,3,3,3,3,4,4,4,5,5,5],[3,5,4,3,0,2,4,5,1,3,5,0,3,4]])
edge_attr = np.array([[1,0,2],[1,0,1],[3,0,1],[2,0,5],[1,0,2],[2,0,5],[6,0,1],[5,0,8],[3,0,1],[6,0,1],[4,0,5],[1,0,1],[0,0,2],[4,0,5]])

r = 0.5

print("Original edge_index \n",edge_index)

num_edge = len(edge_index.T)
edge_index = edge_index.T
new_node = np.max(edge_index)
coarsened_edge = np.sort(np.random.choice(range(0, num_edge), int(num_edge * r/2), replace = False))
mark_del_node_feat = np.full(node_feat.shape[1], -1)

Original edge_index 
 [[0 0 1 2 3 3 3 3 4 4 4 5 5 5]
 [3 5 4 3 0 2 4 5 1 3 5 0 3 4]]


In [43]:
# Check whether coarsened_edge includes some two edges that are same in undirected graph

for idx_c_edge in range(len(coarsened_edge)):
   
  dup_edge = np.array([edge_index[coarsened_edge[idx_c_edge]][1], edge_index[coarsened_edge[idx_c_edge]][0]])
  
  for j in range(len(coarsened_edge)):
    if j != idx_c_edge:
             
       if (edge_index[coarsened_edge[j]] == dup_edge).all():
          coarsened_edge = np.delete(coarsened_edge, j)
          break

  if idx_c_edge+1 == len(coarsened_edge):
    break

print("Which edges will be deleted \n",coarsened_edge)

Which edges will be deleted 
 [1 5]


In [44]:
# Change edge_index & node_feat

for idx_c_edge in coarsened_edge:
  
  new_node += 1

  c_source_node = edge_index[idx_c_edge, 0]
  c_target_node = edge_index[idx_c_edge, 1]

  # print("target edge : ", [c_source_node, c_target_node])
  if c_source_node == -1 or c_target_node == -1:
    continue

  new_node_feat = coarsening_node_feat(node_feat, c_source_node, c_target_node)
  
  # Marking deleted node features by -1 array
  node_feat[c_source_node] = mark_del_node_feat
  node_feat[c_target_node] = mark_del_node_feat
  node_feat = np.vstack((node_feat, new_node_feat))
  
  # Replacing coarsened node index to new node index
  mask_source = np.logical_or(edge_index[:, 0] == c_source_node, edge_index[:, 0] == c_target_node)
  mask_target = np.logical_or(edge_index[:, 1] == c_source_node, edge_index[:, 1] == c_target_node)
  edge_index[mask_source, 0] = new_node
  edge_index[mask_target, 1] = new_node

  # Updating edge features for coarsened edges
  edge_attr[mask_source] = coarsening_edge_attr(edge_attr[mask_source])
  edge_attr[mask_target] = coarsening_edge_attr(edge_attr[mask_target])

  # Marking deleted edge indices and features by [-1, -1] and [-1, -1, ... , -1]
  mask_self_loop = edge_index[:, 0] == edge_index[:, 1]
  edge_index[mask_self_loop] = np.array([-1, -1])
  edge_attr[mask_self_loop] = np.full(edge_attr.shape[1], -1)

  print(edge_attr)

#print("edge_attr \n", edge_attr)

[[ 7  0 11]
 [-1 -1 -1]
 [ 3  0  1]
 [ 2  0  5]
 [24  0 37]
 [ 2  0  5]
 [ 6  0  1]
 [24  0 37]
 [ 3  0  1]
 [ 6  0  1]
 [24  0 37]
 [-1 -1 -1]
 [ 7  0 11]
 [ 7  0 11]]
[[136   0 193]
 [ -1  -1  -1]
 [  3   0   1]
 [ -1  -1  -1]
 [ 58   0  85]
 [ -1  -1  -1]
 [ 58   0  85]
 [ 58   0  85]
 [  3   0   1]
 [136   0 193]
 [ 24   0  37]
 [ -1  -1  -1]
 [136   0 193]
 [  7   0  11]]


In [45]:
# Delete some rows or columns for marking coarsened nodes or edges

rows_to_delete = np.where(np.all(node_feat == mark_del_node_feat, axis=1))
node_feat = np.delete(node_feat, rows_to_delete, axis=0)

rows_to_delete = np.where(np.all(edge_index == np.array([-1,-1]), axis=1))
edge_index = np.delete(edge_index, rows_to_delete, axis=0)
edge_index = np.unique(edge_index, axis = 0).T # Delete duplicated node pairs

rows_to_delete = np.where(np.all(edge_attr == np.full(edge_attr.shape[1], -1), axis=1))
edge_attr = np.delete(edge_attr, rows_to_delete, axis=0)
# edge_attr = np.unique(edge_attr, axis = 0).T # Delete duplicated node pairs

print("Result node_feat \n",node_feat)
print("Result edge_index \n",edge_index)
print("Result edge_attr \n",edge_attr)

Result node_feat 
 [[1 1 1 1]
 [4 4 4 4]
 [5 5 5 5]
 [5 5 5 5]]
Result edge_index 
 [[1 4 4 4 6 6 7 7]
 [4 1 6 7 4 7 4 6]]
Result edge_attr 
 [[  3   7  24  58 136]
 [  0   0   0   0   0]
 [  1  11  37  85 193]]


In [18]:
# Reindex edge_index

flattened_edge_index = edge_index.flatten()
unique_elements = np.unique(flattened_edge_index)
element_mapping = {element: new_value for new_value, element in enumerate(unique_elements)}
edge_index = np.vectorize(element_mapping.get)(edge_index)

print("Reindexed node_feat \n", node_feat)
print("Reindexed edge_index \n", edge_index)

Reindexed node_feat 
 [[ 0  0  0  0]
 [ 2  2  2  2]
 [13 13 13 13]]
Reindexed edge_index 
 [[0 1 2 2]
 [2 2 0 1]]
